# Observer Class

The `Observer` Object Class within SORA was created to deal with the observer location. The online documentation <a href=https://sora.readthedocs.io/>here</a> contains the details about every step.  

This Jupyter-Notebook was designed as a tutorial for how to work with the `Observer` Object Class. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.

**The** `Observer` **Docstring was designed to help the users. Also, each function has its Docstring containing its main purpose and the needed parameters (physical description and formats). Please, do not hesitate to use it.**

## 0. Index

1. [Instantiating the Observer Object](#section_1)
2. [Setting and/or modifing parameters](#section_2)
3. [Apparent Sidereal Time](#section_3)
4. [Ksi and Eta projection](#section_5)
5. [MPC observatories](#section_6)

In [1]:
## import the Observer Class
from sora.observer import Observer
## This means the Observer Class is imported from the "observer" module of the sora package

## To facilitate, sora allows to import Observer directly from the sora package.
from sora import Observer

SORA version: 0.2dev


<a id="section_1"></a>

## 1. Instantiating the Observer Object

The `Observer` Object Class can be instantiated in different ways. First, all observers should have a _ _name_ that will distinguish it from other `Observer` Objects and also allows the `Occultation` Object to control all the steps and Objects. This is the name which the user will use to refer to this observer within the `Occultation` Object. The name is not needed when the site is downloaded from the MPC database in which case the name in the MPC will be used.

The `Observer` main objective is to deal with the observer location, convert the coordinates from ITRS and GCRS and vice-versa and calculate the Orthographic projection the coordinates in the direction of a coordinate.

In [2]:
Observer?

Init signature: Observer(**kwargs)
Docstring:      <no docstring>
Init docstring:
Defines the observer object

Parameters:
    name (str): Name for the Observer.
        Observer is uniquely defined (name must be different for each observer).
    code (str): The IAU code for SORA to search for its coordinates in MPC database
    site (EarthLocation): User provides an EarthLocation object.
    lon (str, float): The Longitude of the site in degrees.
        Positive to East. Range (0 to 360) or (-180 to +180)
        User can provide in degrees (float) or hexadecimal (string)
    lat (str, float): The Latitude of the site in degrees.
        Positive North. Range (+90 to -90)
        User can provide in degrees (float) or hexadecimal (string)
    height (int, float): The height of the site in meters above see level.

Examples:
    User can provide one of the following to define an observer:
    - If user will use the MPC name for the site:
        Observer(code)
    - If user wants to us

**Example of an** `Observer` **intantiated with a MPC code**.

In [3]:
opd = Observer(code='874')

In [4]:
## To see the name of the object "opd"
opd.name

'Observatorio do Pico dos Dias, Itajuba'

In this case, the name is a bit too long, the user can define a differente name upon instantiation.

In [5]:
opd = Observer(name='OPD', code='874')

**Example of** `Observer` **instantiated with coordinates**.

This coordinates must in degrees for longitude and latitude and in meters in height.

In [6]:
# using string representation
obs1 = Observer(name='Ex 1', lon='-45 34 57', lat='-22 32 04', height=1864)

In [7]:
# using numeric representation
obs2 = Observer(name='Ex 2', lon=-69.295805, lat=-31.79877, height=2495)

**Example using an Astropy** `EarthLocation` **Object**.

Astropy has an `EarthLocation` Class with observer location information. SORA uses this class as a core functionality in the `Observer` Class. So the user can give an object create from `EarthLocation` as input.

In [8]:
from astropy.coordinates import EarthLocation
site = EarthLocation('-45 34 57', ' -22 32 04', 1864)
obs3 = Observer(name='Ex 3', site=site)

**The user can access some information from the object as attribute**

In [12]:
obs3.name

'Ex 3'

In [13]:
obs3.lon

<Longitude -45.5825 deg>

In [14]:
obs3.lat

<Latitude -22.53444444 deg>

In [15]:
obs3.height

<Quantity 1864. m>

If the user wants to see the informations in the object, just prints the object directly.

In [16]:
print(obs3)

Site: Ex 3
Geodetic coordinates: Lon: -45d34m57s, Lat: -22d32m04s, height: 1.864 km


<a id="section_2"></a>

## 2. Setting and/or modifing parameters

If any parameter given to `Observer` is not correct, the user does not need to delete or overwrite the previous object. It can be given the new values directly to the object attributes. Only the _name_ attribute cannot be replaced.

In [17]:
print(obs3.lon)

-45d34m57s


In [18]:
obs3.lon = '-35 40 26'
print(obs3.lon)

-35d40m26s


In [19]:
obs3.lon = 67.3514
print(obs3.lon)

67d21m05.04s


In [20]:
obs3.name = 'Teste'

AttributeError: can't set attribute

<a id="section_3"></a>

## 3. Apparent Sidereal Time

If the user, for some reason, wants to know the apparent sidereal time at a certain location for a given time, there exists a function that calculates it.

In [21]:
obs3.sidereal_time?

Signature: obs3.sidereal_time(time, mode='local')
Docstring:
Calculates the Apparent Sidereal Time at a reference time

Parameters:
    time (str,Time): Reference time to calculate sidereal time.
    mode (str): local or greenwich
        If 'local': calculates the sidereal time for the coordinates of this object.
        If 'greenwich': calculates the Greenwich Apparent Sidereal Time.

Returns:
    sidereal_time: An Astropy Longitude object with the Sidereal Time.
File:      ~/anaconda3/lib/python3.8/site-packages/sora/observer/core.py
Type:      method


In [22]:
obs3.sidereal_time('2020-05-10 00:00:00')

<Longitude 19.70677566 hourangle>

The input time can also be an Astropy `Time` Object

In [23]:
from astropy.time import Time

t = Time('2020-05-10 00:00:00')

obs3.sidereal_time(t)

<Longitude 19.70677566 hourangle>

If the user wants to calculate the Greenwich Apparent Sidereal Time, it can be done as:

In [24]:
obs3.sidereal_time('2020-05-10 00:00:00', mode='greenwich')

<Longitude 15.21668233 hourangle>

Also, with the Observer the user can calculate the altitude and azimuth of a given target.

In [25]:
obs3.altaz?

Signature: obs3.altaz(time, coord)
Docstring:
Calculates the Altitude and Azimuth at a reference time for a coordinate

Parameters:
    time (str,Time): Reference time to calculate the sidereal time.
    coord (str, astropy.SkyCoord): Coordinate of the target ICRS.

Returns:
    altitude (float): object altitude in degrees.
    azimuth (float): object azimuth in degrees.
File:      ~/anaconda3/lib/python3.8/site-packages/sora/observer/core.py
Type:      method


In [26]:
altitude, azimuth = obs3.altaz(time='2020-05-10 00:00:00', coord='20 12 19.3 +25 30 00.5')

print('Altitude {:.2f} degrees'.format(altitude))
print('Azimuth  {:.2f} degrees'.format(azimuth))

Altitude 41.33 degrees
Azimuth  9.25 degrees


<a id="section_4"></a>

## 4. Ksi and Eta projection

`Observer` object calculates the orthographic projection (ksi and eta) of a site in the direction of a star given the following function. Ksi is in the East direction and Eta in the North direction.

The coordinates of the star given must be in the Geocentric Celestial Reference System (GCRS).

This is calculated automaticaly in `Occultation`.

In [28]:
obs3.get_ksi_eta?

Signature: obs3.get_ksi_eta(time, star)
Docstring:
Calculates relative position to star in the orthographic projection.

Parameters:
    time (str, Time): Reference time to calculate the position.
        It can be a string in the format "yyyy-mm-dd hh:mm:ss.s" or an astropy Time object
    star (str, SkyCoord): The coordinate of the star in the same reference frame as the ephemeris.
        It can be a string in the format "hh mm ss.s +dd mm ss.ss"
        or an astropy SkyCoord object.

Returns:
    ksi, eta (float): on-sky orthographic projection of the observer relative to a star
        Ksi is in the North-South direction (North positive)
        Eta is in the East-West direction (East positive)
File:      ~/anaconda3/lib/python3.8/site-packages/sora/observer/core.py
Type:      method


In [29]:
ksi, eta = obs3.get_ksi_eta(time='2020-05-10 00:00:00', star='19 21 18.63201 -21 44 25.3924')
print(ksi, eta)

511.2578070035691 -86.82560981731909


A list of times can be given to the time param, as shown below.

In [30]:
times = ['2020-05-10 00:00:00', '2020-05-10 01:00:00', '2020-05-10 02:00:00', '2020-05-10 03:00:00']
ksi, eta = obs3.get_ksi_eta(time=times, star='19 21 18.63201 -21 44 25.3924')
print(ksi)
print(eta)

[ 511.257807   2017.74529991 3385.67428304 4521.31444529]
[ -86.82560982 -213.25930426 -479.77636107 -868.11505453]


<a id="section_5"></a>

## 5. MPC observatories

The user can download and look all the observatories from the MPC database. These are ground-based observatories. The spacecraft observatories are ignored. The function presented returns a Python dictionary that can be searched by the MPC code.

In [31]:
from sora.observer import search_code_mpc
observatories = search_code_mpc()

In [32]:
observatories['874']

('Observatorio do Pico dos Dias, Itajuba',
 <EarthLocation (4126.27235742, -4211.05894345, -2429.98090308) km>)

In [33]:
observatories['511']

('Haute Provence',
 <EarthLocation (4578.31238503, 458.24339505, 4403.08309658) km>)

In [34]:
observatories

{'000': ('Greenwich', <EarthLocation (3980.65908307, 0., 4966.84662601) km>),
 '001': ('Crowborough',
  <EarthLocation (4017.70150872, 10.81285696, 4937.37963307) km>),
 '002': ('Rayleigh',
  <EarthLocation (3966.96894668, 42.92841063, 4981.324997) km>),
 '003': ('Montpellier',
  <EarthLocation (4613.44108454, 314.51286042, 4381.780119) km>),
 '004': ('Toulouse',
  <EarthLocation (4623.91819079, 118.053188, 4377.12407899) km>),
 '005': ('Meudon',
  <EarthLocation (4205.68487713, 163.84501614, 4776.42734587) km>),
 '006': ('Fabra Observatory, Barcelona',
  <EarthLocation (4786.95713426, 177.55185536, 4197.63692567) km>),
 '007': ('Paris',
  <EarthLocation (4202.69234465, 171.49762241, 4778.64693755) km>),
 '008': ('Algiers-Bouzareah',
  <EarthLocation (5106.30535438, 270.78274243, 3799.96646186) km>),
 '009': ('Berne-Uecht',
  <EarthLocation (4324.63496703, 564.87289002, 4638.1812264) km>),
 '010': ('Caussols',
  <EarthLocation (4581.93585771, 556.19867516, 4389.0193045) km>),
 '011': (

**This Jupyter-Notebook was designed as a tutorial for how to work with the** `Observer` **Object Class. More information about the other classes, please refer to their specif Jupyter-Notebook. Any further question, please contact the core team: Altair Ramos Gomes Júnior, Bruno Eduardo Morgado, Gustavo Benedetti Rossi, and Rodrigo Carlos Boufleur.**

**The End**